In [10]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Dict
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [11]:
def compute_customer_metrics(csv_path: str) -> pd.DataFrame:
    csv_file = Path(csv_path)
    if not csv_file.exists():
        raise FileNotFoundError(f"CSV file not found: {csv_file}")

    df = pd.read_csv(csv_file)
    df.columns = [c.strip() for c in df.columns]

    for col in ["Total Price", "Add-on Total", "Unit Price", "Quantity"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    completed = df[df["Order Status"].eq("Completed")].copy()

    if not completed.empty:
        preferred_payment = (
            completed.groupby("Customer ID")["Payment Method"]
            .agg(lambda s: s.value_counts().idxmax())
            .rename("preferred_payment_method")
        )
    else:
        preferred_payment = pd.Series(dtype=object, name="preferred_payment_method")

    total_spend = (
        completed.groupby("Customer ID")["Total Price"].sum().rename("total_spend")
    )

    addons_spend = (
        completed.groupby("Customer ID")["Add-on Total"].sum().rename("addons_spend")
    )

    all_customers = pd.Index(df["Customer ID"].unique(), name="Customer ID")
    result = (
        pd.DataFrame(index=all_customers)
        .join(preferred_payment)
        .join(total_spend)
        .join(addons_spend)
        .sort_index()
    )

    for col in ["total_spend", "addons_spend"]:
        if col in result.columns:
            result[col] = result[col].fillna(0.0)

    return result


In [12]:
metrics = compute_customer_metrics("Electronic_sales_Sep2023-Sep2024.csv")

metrics.to_csv("customer_metrics.csv", index=True, float_format="%.2f")

print(f"Computed metrics for {len(metrics)} customers.")
print(metrics.head(10))
print(f"Saved to: customer_metrics.csv")

Computed metrics for 12136 customers.
            preferred_payment_method  total_spend  addons_spend
Customer ID                                                    
1000                          Paypal       741.09         26.09
1002                     Credit Card      5020.60         60.16
1003                            Cash        41.50         35.56
1004                     Credit Card        83.00         65.78
1005                          Paypal     11779.11         75.33
1006                            Cash      6645.94         90.38
1007                             NaN         0.00          0.00
1008                            Cash      3379.32         65.85
1011                     Credit Card      7911.90         70.17
1013                             NaN         0.00          0.00
Saved to: customer_metrics.csv


In [18]:
def compute_revenue_breakdowns(csv_path: str) -> Dict[str, pd.DataFrame]:

    csv_file = Path(csv_path)
    if not csv_file.exists():
        raise FileNotFoundError(f"CSV file not found: {csv_file}")

    df = pd.read_csv(csv_file)
    df.columns = [c.strip() for c in df.columns]

    for col in ["Total Price", "Add-on Total", "Unit Price", "Quantity"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    if "Purchase Date" in df.columns:
        df["Purchase Date"] = pd.to_datetime(df["Purchase Date"], errors="coerce")

    completed = df[df["Order Status"].eq("Completed")].copy()

    shipping_rev = (
        completed.groupby("Shipping Type")["Total Price"].sum().sort_values(ascending=False)
    ).to_frame("revenue")

    product_rev = (
        completed.groupby("Product Type")["Total Price"].sum().sort_values(ascending=False)
    ).to_frame("revenue")

    if completed["Purchase Date"].notna().any():
        monthly = (
            completed.groupby(completed["Purchase Date"].dt.to_period("M"))["Add-on Total"]
            .sum()
            .sort_index()
            .to_frame("addons_revenue")
        )
        monthly.index = monthly.index.astype(str)

        quarterly = (
            completed.groupby(completed["Purchase Date"].dt.to_period("Q"))["Add-on Total"]
            .sum()
            .sort_index()
            .to_frame("addons_revenue")
        )
        quarterly.index = quarterly.index.astype(str)
    else:
        monthly = pd.DataFrame({"addons_revenue": []})
        quarterly = pd.DataFrame({"addons_revenue": []})

    return {
        "revenue_by_shipping_type": shipping_rev,
        "revenue_by_product_type": product_rev,
        "addons_revenue_by_month": monthly,
        "addons_revenue_by_quarter": quarterly,
    }

In [19]:
def export_breakdowns(breakdowns: Dict[str, pd.DataFrame]) -> None:
    for name, frame in breakdowns.items():
        frame.to_csv(f"{name}.csv")

In [20]:
def plot_breakdowns(breakdowns: Dict[str, pd.DataFrame]) -> None:
    ship = breakdowns["revenue_by_shipping_type"]["revenue"]
    fig, ax = plt.subplots(figsize=(8, 5))
    ship.plot(kind="bar", ax=ax, color="#4C78A8")
    ax.set_title("Revenue by Shipping Type (Completed Orders)")
    ax.set_xlabel("Shipping Type")
    ax.set_ylabel("Revenue")
    ax.tick_params(axis="x", rotation=30)
    fig.tight_layout()
    fig.savefig("revenue_by_shipping_type.png", dpi=160)
    plt.close(fig)

    prod = breakdowns["revenue_by_product_type"]["revenue"]
    fig, ax = plt.subplots(figsize=(8, 5))
    prod.plot(kind="bar", ax=ax, color="#F58518")
    ax.set_title("Revenue by Product Type (Completed Orders)")
    ax.set_xlabel("Product Type")
    ax.set_ylabel("Revenue")
    ax.tick_params(axis="x", rotation=30)
    fig.tight_layout()
    fig.savefig("revenue_by_product_type.png", dpi=160)
    plt.close(fig)

    monthly = breakdowns["addons_revenue_by_month"]["addons_revenue"]
    if not monthly.empty:
        fig, ax = plt.subplots(figsize=(10, 4))
        monthly.plot(ax=ax, marker="o", color="#54A24B")
        ax.set_title("Add-on Revenue by Month")
        ax.set_xlabel("Month")
        ax.set_ylabel("Add-on Revenue")
        ax.tick_params(axis="x", rotation=45)
        fig.tight_layout()
        fig.savefig("addons_revenue_by_month.png", dpi=160)
        plt.close(fig)

    quarterly = breakdowns["addons_revenue_by_quarter"]["addons_revenue"]
    if not quarterly.empty:
        fig, ax = plt.subplots(figsize=(8, 4))
        quarterly.plot(ax=ax, marker="o", color="#E45756")
        ax.set_title("Add-on Revenue by Quarter")
        ax.set_xlabel("Quarter")
        ax.set_ylabel("Add-on Revenue")
        ax.tick_params(axis="x", rotation=0)
        fig.tight_layout()
        fig.savefig("addons_revenue_by_quarter.png", dpi=160)
        plt.close(fig)

In [21]:
breakdowns = compute_revenue_breakdowns("Electronic_sales_Sep2023-Sep2024.csv")
export_breakdowns(breakdowns)
plot_breakdowns(breakdowns)
print("Saved breakdown CSVs and charts.")

Saved breakdown CSVs and charts.
